In [1]:
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote
import unicodedata, re
from tqdm.notebook import tqdm
import string

In [2]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver.exe')  # 윈도우 ## open driver for window version
driver.get(insta_url)
time.sleep(1)
# with open('D:\WorkSpace\mykey\instar_pw.txt') as f:
#     password = f.read()
# with open('D:\WorkSpace\mykey\instar_id.txt') as f:
#     id = f.read()
id = '01056537972'
password = '931811ji!!!'
input_id = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_id.clear()
input_id.send_keys(id)
time.sleep(2)
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(3)
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

In [3]:
df_name= pd.read_excel("제주골프장_정보(강지희).xlsx")

In [4]:
golf_course_name=list(df_name.골프장명)
golf_course_name.append('제주CC')
type(golf_course_name)

list

In [5]:
keyword = golf_course_name[0]
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EA%B7%B8%EB%A6%B0%ED%95%84%EB%93%9CCC'

In [6]:
golf_course_name[0]

'그린필드CC'

In [7]:
driver.get(url)
time.sleep(7)

In [27]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

C:\Users\alsdn\AppData\Local\Temp/ipykernel_21848/2042192687.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('div._9AhH0').click()


In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [31]:
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#골프',
 '#골린이',
 '#골린이성장일기⛳️',
 '#구력6개월차',
 '#언제싱글처보노',
 '#드라이버',
 '#드라이버스윙',
 '#제골조',
 '#제주골프',
 '#그린필드cc',
 '#비거리280m40주']

In [30]:
date = soup.select_one('._1o9PC')['datetime'][:10]
date

TypeError: 'NoneType' object is not subscriptable

In [160]:
try:
    like = soup.select_one('.EDfFK.ygqzn').text[:]
except:
    like = 0
like

'좋아요 47개'

In [31]:
row = [content, date, like, tags]
row

NameError: name 'content' is not defined

In [139]:
# 첫번째 게시글에서 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button/div').click()
time.sleep(2)

C:\Users\alsdn\AppData\Local\Temp/ipykernel_4356/1498655362.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button/div').click()


In [ ]:
# 두번째 게시글부터 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [ ]:
# 5를 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [5]:
def get_content(driver,soup):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    try:
        content = soup.select_one('.C4VMK').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = '내용이없습니다'    
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('._1o9PC')['datetime'][:10]
    try:
        like = soup.select_one('.EDfFK.ygqzn').text[:]
    except:
        like = 0
    return [content, date, like, tags]

In [7]:
게시글수=[]
results =[]
for i,keyword in enumerate(golf_course_name):
    search_url = 'https://www.instagram.com/explore/tags/'
    url = f'{search_url}{quote(keyword)}'
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.get(url)
    time.sleep(7)
    try:
        게시글수.append(int(driver.find_element_by_xpath('/html/body/div[1]/section/main/header/div[2]/div/div[2]/div/span').text))
        driver.find_element_by_xpath('/html/body/div[1]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]').click()
        for j in tqdm(range(100)):
            if j == 0:
                row = get_content(driver,soup)
                row.append(keyword)
                results.append(row)
                driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div/button').click()
                time.sleep(3)
            try:
                row = get_content(driver,soup)
                row.append(keyword)
                results.append(row)
                driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/div[2]/button').click()
                time.sleep(3)
            except:
                pass
    except:
        results.append(['내용이없습니다','내용이없습니다','내용이없습니다','내용이없습니다','내용이없습니다',])

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
results

[['jeju_kingkong_golf#골프 #골린이 #골린이성장일기⛳️ #구력6개월차 #언제싱글처보노#드라이버 #드라이버스윙 #제골조 #제주골프 #그린필드cc #비거리280m40주',
  '2021-06-07',
  '좋아요 47개',
  ['#골프',
   '#골린이',
   '#골린이성장일기⛳️',
   '#구력6개월차',
   '#언제싱글처보노',
   '#드라이버',
   '#드라이버스윙',
   '#제골조',
   '#제주골프',
   '#그린필드cc',
   '#비거리280m40주']],
 '그린필드CC',
 ['sxx_ahㅈHㅁI없는 골푸 🏌🏻\u200d♀️🥊늘 골프가 1순위였능데 이젠 ,,,, 안룡 ~#골스타그램 #그린필드cc #스윙 #티샷수정됨\xa0·\xa038주',
  '2021-06-22',
  '1st_won_yi님 여러 명이 좋아합니다',
  ['#골스타그램', '#그린필드cc', '#스윙', '#티샷수정됨']],
 '그린필드CC',
 ['y_s_w99오늘 바람 아예없고 그늘도 없고 진짜진짜 더운날 지금 현재 상태 살짝  더위먹은듯..어질어질하네예- 그린필드cc 어려운데 재밋뜨아 #제주골프 #그린필드cc #조인골프 #골프 #필드 #라운딩 #그린이어려운골프장 #오늘도 #굿샷32주',
  '2021-08-07',
  '좋아요 70개',
  ['#제주골프',
   '#그린필드cc',
   '#조인골프',
   '#골프',
   '#필드',
   '#라운딩',
   '#그린이어려운골프장',
   '#오늘도',
   '#굿샷32주']],
 '그린필드CC',
 ['jawoniing3박4일 제주 전지훈련 끝🙋🏻\u200d♀️⛳️수정됨\xa0·\xa010주',
  '2022-01-05',
  '좋아요 145개',
  []],
 '그린필드CC',
 ['jeju_kingkong_golf#골린이성장일기⛳️ #라베 #싱글 #골프 #그린필드cc #구력7개월차 #굿샷⛳️ #제주골퍼 #제골조밴드32주',
  '2021-08-04',
  '좋아요 44개',
  

In [ ]:
df = pd.DataFrame(results, columns=['content','date','like','tags','골프장검색키워드'])
df.head()
df.to_csv('골프장데이터sns검색.csv',index=False)
df.to_csv('G:\\내 드라이브\\넥스트이지!!\넥스트이지 골프장합친거\\골프장데이터sns검색.csv',index=False)

In [ ]:
G:\\내 드라이브\\넥스트이지!!\넥스트이지 골프장합친거\\